<a href="https://colab.research.google.com/github/StefanHubner/MachineLearningEconomics/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This takes about 9-10 minutes

Sys.setenv("CUDA" = "cpu")
Sys.setenv("TORCH_INSTALL" = "1")
install.packages("torch", reinstall = FALSE)
torch::install_torch()

In [ ]:
library(torch)

x1x2 <- cbind(1, as.matrix(train[,c("balance", "income")])/1000)
y1 <- train$default1

Sys.setenv("CUDA" = "cuda")
# by default it will use the CPU, cuda refers to the GPU
dev <- ifelse(cuda_is_available(), "cuda", "cpu")
cat(paste0("Running on: ", dev))

In [ ]:
y <- torch_tensor(y1, dtype = torch_float64(), requires_grad = TRUE)
x <- torch_tensor(x1x2, dtype = torch_float64(), requires_grad = TRUE)
beta <- torch_randn(3, 1, dtype = torch_float64(), requires_grad = TRUE)

# Define loss function: likelihood (sometimes called binary cross-entropy in ML context)
ll <- function(beta) {
  xbhat <- torch_mm(x, beta)
  yhat <- torch_sigmoid(xbhat)
  loss <- -torch_sum(y * torch_log(yhat) + (1 - y) * torch_log(1 - yhat))
  loss
}

optimizer <- optim_lbfgs(beta, lr = .2)

# A 'closure', i.e. the code that is run in each step of the optimiser
calc_loss <- function() {
  optimizer$zero_grad()
  value <- ll(beta)
  value$backward()
  nn_utils_clip_grad_value_(beta, 20)
  value
}

In [ ]:
for (i in 1:100) {
  logl <- optimizer$step(calc_loss) # 20 internal iterations per step
  display_html(sprintf("<tt>%d: %f</tt>", i, as.numeric(logl$detach())))
}

In [ ]:
as.numeric(beta$detach())

In [ ]:
# debugging

debugstart <- t(t(c(1, 2, 3)))
x0 <- t(c(1, 1/2, 1/3))

y <- torch_tensor(y1[1], dtype = torch_float64(), requires_grad = TRUE)
x <- torch_tensor(x0, dtype = torch_float64(), requires_grad = TRUE)

beta <- torch_tensor(debugstart, dtype = torch_float64(), requires_grad = TRUE)
# Define loss function: likelihood (sometimes called binary cross-entropy in ML context)

xbhat <- torch_mm(x, beta)
xbhat$retain_grad()

yhat <- torch_sigmoid(xbhat)
yhat$retain_grad()

loss <- -torch_sum(y * torch_log(yhat) + (1 - y) * torch_log(1 - yhat))
loss$backward()

y
x
xbhat
xbhat$grad # dloss/dp
yhat
yhat$grad # dloss/dyhat

loss <- -torch_sum(y * torch_log(yhat) + (1 - y) * torch_log(1 - yhat))
